In [13]:
from pydantic import validate_arguments, ValidationError, validate_call

class Coin:
    def __init__(self, coin_name: str):
        self._symbol: str = coin_name
        self._stats_obj = None

    @property
    def symbol(self):
        return self._symbol

    @validate_call
    def set_symbol(self, value: str):
        self._symbol = value

    @symbol.setter
    def symbol(self, value):
        try:
            self.set_symbol(value)
        except ValidationError as e:
            raise ValueError(f"Invalid value for symbol: {e}")

# Usage
coin = Coin("BTC")

# Valid update
coin.symbol = "ETH"
print(coin.symbol)  # Output: ETH

# Invalid update
try:
    coin.symbol = "OOO"
except ValueError as e:
    print(e)  # Output: Invalid value for symbol: 1 validation error for SetSymbol -> value str type expected (type=type_error.str)

ETH


In [6]:
coin.symbol

'OOO'

In [51]:
from pydantic import validate_arguments, ValidationError, validate_call

class Coin:
    @validate_call
    def __init__(self, coin_name: str):
        self._symbol: str = self.set_symbol(coin_name)
        self._stats_obj = None

    @property
    def symbol(self):
        return self._symbol

    @validate_call
    def set_symbol(self, value: str):
        print("Setting symbol", value)
        self._symbol = value

    @symbol.setter
    def symbol(self, value):
        try:
            self.set_symbol(value)
        except ValidationError as e:
            raise ValueError(f"Invalid value for symbol: {e}")

# Usage
coin = Coin('BTC')

# Valid update
coin.symbol = "ETH"
print(coin.symbol)  # Output: ETH

Setting symbol BTC
Setting symbol ETH
ETH


In [46]:
from pydantic import validate_arguments, ValidationError, validate_call

class Coin:
    @validate_call
    def __init__(self):
        self._symbol: str = None
        self._stats: str = None

    @property
    def symbol(self):
        return self._symbol

    @symbol.setter
    @validate_call
    def symbol(self, value):
        print("Setting symbol", value)
        try:
            if self._symbol is not None:
                raise ValueError("Symbol already set. You have to create a new object for a different symbol.")
            else:
                print("Else")
                self._symbol = value
                self.__setattr__("stats", "")
        except ValidationError as e:
            raise ValueError(f"Invalid value for symbol: {e}")

    @property
    def stats(self):
        return self._stats

    @stats.setter
    @validate_call
    def stats(self, value):
        print("Setting stats")
        try:
            self._stats = "Hello World!"
        except ValidationError as e:
            raise ValueError(f"Invalid value for stats: {e}")

# Usage
coin = Coin()

# Valid update
coin.symbol = "ETH"
# coin.symbol = "BTC"

Setting symbol ETH
Else
Setting stats


In [43]:
coin.stats

'Hello World how you doing!'

In [80]:
from pydantic import validate_arguments, ValidationError, validate_call
import random

class Stats:
    def __init__(self, symbol):
        self.symbol: str = symbol
        self.price: float = 0.0
        self.refresh()

    def refresh(self):
        self.price = random.randint(1, 100)


class Coin:
    @validate_call
    def __init__(self, coin_name: str):
        self.symbol: str = self.set_symbol(coin_name)
        self.stats_obj: Stats = Stats(self._symbol)

    @validate_call
    def set_symbol(self, value: str):
        print("Setting symbol", value)
        self._symbol = value

    def refresh_stats(self):
        self.stats_obj.refresh()


# Usage
coin = Coin('BTC')

# Valid update
coin.symbol = "ETH"
print(coin.stats_obj.price)  # Output: ETH

Setting symbol BTC
83


In [87]:
coin.stats_obj.refresh()
print(coin.stats_obj.price)  # Output: ETH

60


In [265]:
from pydantic import (
    validate_arguments,
    ValidationError,
    validate_call,
    BaseModel,
    field_validator,
    Field,
)
import random


class Stats(BaseModel):
    symbol: str
    price: float = None

    def __init__(self, **data):
        super().__init__(**data)
        self.refresh()

    def refresh(self):
        self.price = random.randint(1, 100)


class Coin(BaseModel):
    symbol: str
    stats_obj: Stats | None = None

    def __init__(self, **data):
        super().__init__(**data)
        self.stats_obj = Stats(symbol=self.symbol)

    @field_validator("symbol")
    def validate_symbol(cls, value):
        if value not in ["BTC", "ETH"]:
            raise ValueError("Symbol must be BTC")
        return value

    class Config:
        validate_assignment = True

    def refresh_stats(self):
        self.stats_obj.refresh()


# Usage
coin = Coin(symbol="BTC")

# Valid update
coin.symbol = "ETH"
# print(coin.stats_obj.price)
coin


Stats object creating....
Validating stats object
symbol='BTC' price=23
Stats object created....


Coin(symbol='ETH', stats_obj=Stats(symbol='BTC', price=23))

In [244]:
coin.stats_obj.refresh()
coin.stats_obj.price

72

In [262]:
coin.symbol = "ETH5"

ValidationError: 1 validation error for Coin
symbol
  Value error, Symbol must be BTC [type=value_error, input_value='ETH5', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

In [264]:
coin.stats_obj = "hello"

ValidationError: 1 validation error for Coin
stats_obj
  Input should be a valid dictionary or instance of Stats [type=model_type, input_value='hello', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

In [287]:
import requests

def get_response(method="get", endpoint_method="", params={}, headers={}):
    # url = "http://binance_api:8000/"
    url = "http://localhost:8000/"
    if endpoint_method != "":
        url += endpoint_method

    if not headers:
        headers = {"accept": "application/json"}

    if not params:
        params = {}
    if method == "get":
        response = requests.get(url, headers=headers, params=params)
    elif method == "post":
        response = requests.post(url, headers=headers, params=params)
    else:
        raise ValueError("Invalid method")

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError("Error in response")


def get_coin_price(self):
    return get_response(method="get", endpoint_method="coinpair_price", params={"coin_list": ["btc", "eth", "doge"]})

get_coin_price('btc')

{'isSuccess': True, 'price': None, 'price_list': [69140.0, 3735.27, 0.16551]}

In [277]:
import requests

url = "http://localhost:8000/coinpair_price/"

payload = {'coin_name': 'btc'}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload, timeout=400000)

print(response.text)


{"detail":"Either coin_name or coin_list must be provided."}


In [1]:
from typeguard import typechecked

@typechecked
def add_numbers(a: int, b: int) -> int:
    return a + b

# Usage
add_numbers(1, 2)  # Works fine
# add_numbers(1, 'two')  # Raises TypeError at runtime


TypeError: <module '__main__'> is a built-in module

In [17]:
from pydantic import BaseModel, field_validator

class Stats(BaseModel):
    symbol: str = None
    market_cap: float = 0
    price: float = 0
    volume: float = 0

    @field_validator('symbol')
    def symbol_must_be_positive(cls, value):
        return 'BTC'

    @field_validator('market_cap')
    def market_cap_must_be_positive(cls, value):
        return 10000000

    @field_validator('price')
    def price_must_be_positive(cls, value):
        return 1.2

    @field_validator('volume')
    def volume_must_be_positive(cls, value):
        return 562


class Coin(BaseModel):
    symbol: str
    stats_obj: Stats = Stats(symbol=cls.symbol, market_cap=0, price=0, volume=0)

    @field_validator('symbol')
    def symbol_must_be_positive(cls, value):
        if value not in ['BTC', 'ETH']:
            raise ValueError('Symbol must be BTC')

obj = Coin()

NameError: name 'cls' is not defined

In [14]:
obj

Coin(symbol='', stats_obj=Stats(market_cap=10000000, price=1.2, volume=562))

In [169]:
from pydantic import BaseModel, validator, field_validator, Field, root_validator, model_validator
from typing import Any
from typing import List, Annotated

def get_market_cap(symbol: str) -> float:
    return 10000000


class Stats(BaseModel):
    symbol: str
    market_cap: float = None
    price: float = None
    volume: float = None

    def get_market_cap(self):
        return 10000000

    def get_price(self):
        print("Getting price", self.symbol)
        return 1.2

    def get_volume(self):
        print("Getting volume", self.symbol)
        return 6365555

    def refresh(self):
        print("Refreshing stats")
        print(self.symbol)
        self.market_cap = self.get_market_cap()
        self.price = self.get_price()
        self.volume = self.get_volume()

    def __init__(self, **data):
        super().__init__(**data)
        self.refresh()


class Coin(BaseModel):
    symbol: str
    stats_obj: Stats | None = None

    @field_validator("symbol")
    def symbol_must_be_btc_or_eth(cls, value):
        if value not in ["BTC", "ETH"]:
            raise ValueError("Symbol must be either BTC or ETH")
        return value

    @field_validator("stats_obj", mode="after")
    def create_stats_obj(cls, v: Any, values: dict):
        return Stats(symbol=values.data["symbol"])


coin = Coin(symbol="BTC", stats_obj=None)
print(coin)


Refreshing stats
BTC
Getting market cap BTC
Getting price BTC
Getting volume BTC
symbol='BTC' stats_obj=Stats(symbol='BTC', market_cap=typing.Annotated[float, 'apple'], price=1.2, volume=6365555)


In [170]:
coin

Coin(symbol='BTC', stats_obj=Stats(symbol='BTC', market_cap=typing.Annotated[float, 'apple'], price=1.2, volume=6365555))

In [154]:
coin.stats_obj.refresh()

Refreshing stats
BTC


In [155]:
coin

Coin(symbol='BTC', stats_obj=Stats(symbol='BTC', market_cap=787, price=898, volume=696))

In [70]:
from pydantic import BaseModel, field_validator, root_validator, model_validator

class ExampleModel(BaseModel):
    x: int
    y: int = 0  # Dependent attribute

    # @model_validator(mode='after')
    def verify_square(self):
        if self.x != self.y:
            self.y = self.x
        return self


# Usage
example = ExampleModel(x=5)
print(example)  # Output will show y as 10


x=5 y=0


In [71]:
example.x = 10

In [73]:
example.verify_square()

ExampleModel(x=10, y=10)

In [74]:
example

ExampleModel(x=10, y=10)

: 

In [54]:
pip install rx

  Obtaining dependency information for rx from https://files.pythonhosted.org/packages/e2/a9/efeaeca4928a9a56d04d609b5730994d610c82cf4d9dd7aa173e6ef4233e/Rx-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/199.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/199.2 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.2 kB ? eta -:--:--
   -------- ------------------------------ 41.0/199.2 kB 393.8 kB/s eta 0:00:01
   ------------ -------------------------- 61.4/199.2 kB 550.5 kB/s eta 0:00:01
   ---------------------------------------- 199.2/199.2 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\sudhi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [55]:
import rx
from rx import operators as ops

class RxReactiveModel:
    def __init__(self):
        self._base_subject = rx.subject.ReplaySubject(1)
        self.squared = self._base_subject.pipe(ops.map(lambda x: x**2))
        self.cubed = self._base_subject.pipe(ops.map(lambda x: x**3))

    def update_base(self, value):
        self._base_subject.on_next(value)

# Usage
model = RxReactiveModel()

model.squared.subscribe(lambda x: print(f"Squared: {x}"))
model.cubed.subscribe(lambda x: print(f"Cubed: {x}"))

model.update_base(2)
model.update_base(3)


Squared: 4
Cubed: 8
Squared: 9
Cubed: 27


In [56]:
model.update_base(10)

Squared: 100
Cubed: 1000


In [52]:
example.__dict__

{'x': 15, 'y': 0}

In [7]:
obj = DBSyncResponse(
    transaction_list=[
        {
            "transaction_id": "1",
            "transaction_date": "2021-10-10",
            "transaction_amount": 1000,
        }
    ],
    isSuccess=True,
    message="Success",
    error="",
)

DBSyncResponse(
    transaction_list=[
        {
            "transaction_id": "1",
            "transaction_date": "2021-10-10",
            "transaction_amount": 1000,
        }
    ],
    isSuccess=True,
    message="Success",
    error="",
).model_dump(mode="json")

{'transaction_list': [{'transaction_id': '1',
   'transaction_date': '2021-10-10',
   'transaction_amount': 1000}],
 'isSuccess': True,
 'message': 'Success',
 'error': ''}

In [8]:
DBSyncResponse(
    transaction_list=[
        {
            "transaction_id": "1",
            "transaction_date": "2021-10-10",
            "transaction_amount": 1000,
        }
    ],
    isSuccess=True,
    message="Success",
    error="",
).model_dump(mode="json")

{'transaction_list': [{'transaction_id': '1',
   'transaction_date': '2021-10-10',
   'transaction_amount': 1000}],
 'isSuccess': True,
 'message': 'Success',
 'error': ''}

In [4]:
import requests
import json

# Define the URL for the request
url = 'http://127.0.0.1:8001/binanalyzer_database/sync_database'

# Define the headers to be sent with the request
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

# Define the payload (data) to be sent in JSON format
data = json.dumps([
    {"name": "sudhir"}
])

# Make the POST request and store the response
response = requests.post(url, headers=headers, data=data)

# Print the status code and the content of the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())


Status Code: 400
Response Body: {'detail': "1 validation error for DBSyncRequest\ntransaction_list\n  Value error, Columns missing in the transaction list: {'time', 'timestamp', 'month', 'usdt_equivalent', 'order_id', 'quote_id', 'coin', 'miscellaneous', 'buy_sell', 'year', 'day', 'order_status', 'trade_type', 'automatically_added', 'coin_amount', 'timezone', 'conversion_ratio'} [type=value_error, input_value=[{'name': 'sudhir'}], input_type=list]\n    For further information visit https://errors.pydantic.dev/2.7/v/value_error"}
